### 1. Show Mr. Pepys that C (at least 3 ⚅s in 18 rolls) is actually the least likely of the three options.


In [3]:
import scipy

print(scipy.stats.binom.pmf(3, 18, 1/6))
print(scipy.stats.binom.pmf(2, 12, 1/6))
print(scipy.stats.binom.pmf(1, 6, 1/6))

0.24519844796019247
0.29609356863138386
0.4018775720164608


### 2. About 10% of passengers who are scheduled to take a particular flight fail to show up. For this reason, airlines overbook flights, selling more tickets than they have seats, with the expectation that they will have some no shows. An airline with seating for 100 passengers sells 110 tickets for the flight. What is the probability that they will have enough seats for all the passengers for all of the passengers who show up for the flight? (Assume that passengers independently show up for the flight. Can you think of a situation where this would not be a reasonable assumption?)

- Let X be count of passengers who fail to show up
- X~Binomial(n=110, p=0.1) assuming independence
- Probability of enough seats means $P(X >= 10)$
- $$\begin{align}
    P(X >= 10) &= 1 - P(X <= 9) \\
    &= 1 - F(9) \\
    &= 1 - f(9) - f(8) - ... f(0)
    \end{align}$$

- From computation below, there will be enough seats ~0.671 of the time

In [40]:
import scipy
import numpy as np

prob=1 - scipy.stats.binom.cdf(9, 110, 0.1)
print(prob)

population = [1]*1 + [0]*9
samples=[np.sum(np.random.choice(population, size=110, replace=True))>=10 for _ in range(25_000)]
np.mean(samples)

0.6710143945558992


0.66724

### 3. In the World Series of baseball, two teams (call them A and B) play a sequence of games against each other, and the first team to win four games wins the series. Suppose team A is slightly better, with a 0.6 probability of winning each game, and assume the games are independent. What is the probability that team A wins the series? (Hint: After 7 games, one of the teams must have won the Series. Even though the teams only play until one team has won four games, this calculation is easiest if you assume that the teams always play 7 games.)

- This logic is wrong
    - There are 4 ways for team A to win. Summing the probabilities gives the probaility of an A win
        - 4-0,4-1,4-2,4-3
        - P(4-0):0.6^4
        - P(4-1):5c1 * 0.6^4 * 0.4^1
        - P(4-2):6c2 * 0.6^4 * 0.4^2
        - P(4-3):7c3 * 0.6^4 * 0.4^3
        - Total prob: 0.9901
    - This is wrong because P(4-1) above is actually P(4-1|5 games played). Obviously, you cannot add that probability to P(4-0|4 games played), because they have completely different denominators. 
    - As an obvious indicator of why this is wrong, think about what happens if there are 9 games instead of 7. The probability exceeds 1!
        - 5-0,5-1,5-2,5-3,5-4
        - P(5-0):5c0 * 0.6^5 * 0.4^0
        - P(5-1):6c1 * 0.6^5 * 0.4^1
        - P(5-2):7c2 * 0.6^5 * 0.4^2
        - P(5-3):8c3 * 0.6^5 * 0.4^3
        - P(5-4):9c4 * 0.6^5 * 0.4^4
        - Total prob: 1.055172096

In [115]:
from math import comb
# print(0.6**4 + (comb(5,1) * 0.6**4 * 0.4**1) + (comb(6,2) * 0.6**4 * 0.4**2) + (comb(7,3) * 0.6**4 * 0.4**3))
np.sum([(comb(5+x,x) * 0.6**5 * 0.4**x) for x in range(5)])

1.055172096

- Instead, the right way to think about this is to disregard situations where fewer than 7 games are played
    - We can do this, because the game is a "first to 4"
    - The outcome of the other 3 games do not matter
    - The order of the other 3 games do not matter
    - For example, if B wins the first 3 games, and A wins the latter 4, A will still win the game overall
    - It is impossible for B to hit 4 games won if A eventually wins, and vice versa
- So we can assume that all 7 games are played
    - A wins the game if they have 4,5,6,7 wins
    - P(4-3),P(5-2)+P(6-1)+P(7-0)
    - This matches our simulation results

In [118]:
import scipy
np.sum(scipy.stats.binom(7, 0.6).pmf([4,5,6,7]))

0.710208

In [123]:
import numpy
population = [1]*6 + [0]*4
samples=[np.sum(np.random.choice(population,size=7,replace=True))>= 4 for _ in range(10_000)]
np.mean(samples)

0.7078

### 4. In the carnival game chuck-a-luck, three dice are rolled. You can make a bet on a particular number (1, 2, 3, 4, 5, 6) showing up. The payout is 1 to 1 if that number shows on (exactly) one die, 2 to 1 if it shows on two dice, and 3 to 1 if it shows up on all three. (You lose your initial stake if your number does not show on any of the dice.) If you make a $1 bet on the number three, what is the distribution of the amount you win? (Hint: The random variable is not binomial but very closely related to a binomial. You can should be able to write the p.m.f. as a table.)

- Let the number you bet on be $N$
- For each dice, there is a $\frac{1}{6}$ probability of getting the number
- The amount you win is simply the $pmf(1)*1 + pmf(2)*2 + pmf(3)*3$

In [133]:
import scipy
import numpy as np

scipy.stats.binom(3,1/6).pmf([0,1,2,3])*[-1,1,2,3]

array([-0.5787037 ,  0.34722222,  0.13888889,  0.01388889])